# Clustering Crypto

### Fetching Cryptocurrency Data

In [16]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [17]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
r = requests.get(url)
json = r.json()
json.keys()
json["Data"]
df = pd.DataFrame(json["Data"]).T
#crypto_df1=crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','MaxSupply']]
df.tail()

NameError: name 'requests' is not defined

In [19]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
# crypto_df = pd.read_csv(file_path)
# crypto_df

### Data Preprocessing

In [8]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df=df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','MaxSupply']]
crypto_df = crypto_df.rename(columns={"MaxSupply":"TotalCoinSupply"})
print(crypto_df.shape)
crypto_df.head(20)

In [9]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df[crypto_df["IsTrading"]==True]
print(crypto_df.shape)

In [10]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df[crypto_df["Algorithm"]!="N/A"]
print(crypto_df.shape)
crypto_df

In [11]:
# Remove the "IsTrading" column
crypto_df.copy().drop(columns=['IsTrading'],  axis=1, inplace=True)
crypto_df

In [12]:
# Remove rows with at least 1 null value
for column in crypto_df.columns:
    print(f"Column {column} has {crypto_df[column].isnull().sum()} null values")

In [16]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df[ crypto_df['TotalCoinsMined'] > 0]
print(crypto_df.shape)

In [17]:
# Drop rows where there are 'N/A' text values
df2 = crypto_df[crypto_df.iloc[:]!='N/A'].dropna()
print(df2.shape)
df2.dropna()

In [18]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
y = df2[["CoinName"]]
y

In [19]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
df2.drop(columns=['CoinName'],  inplace=True)
df2

In [20]:
# Create dummy variables for text features
X = pd.get_dummies(df2, columns=["Algorithm","ProofType"])

X.head()

In [21]:
# Standardize data
crypto_scaled = StandardScaler().fit_transform(X)
print(crypto_scaled[0:5])

### Reducing Dimensions Using PCA

In [22]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)

In [23]:
# Create a DataFrame with the principal components data
pcs_df= pd.DataFrame(
    data=crypto_pca, columns=["principal component 1", "principal component 2", "principal component 3"],index= 
    y.index
)
pcs_df

### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [24]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

Running K-Means with `k=<your best value for k here>`

In [ ]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(pcs_df)
# Predict clusters
predictions = model.predict(pcs_df)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
pcs_df["Class"] = model.labels_
clustered_df = pd.concat([df2,pcs_df,y], join="outer", axis=1)
clustered_df.head()

### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="principal component 1",
    y="principal component 2",
    z="principal component 3",
    color="CoinName",
    symbol="CoinName",
    hover_name="CoinName",
    hover_data=["Algorithm"],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos
clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"],
                          title="Total Number of total tradable cryptos.")

In [ ]:
# Print the total number of tradable cryptocurrencies
print(len(clustered_df["CoinName"]))

#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot
c_scaler = MinMaxScaler()

clustered_scaled = c_scaler.fit_transform(
    clustered_df[["TotalCoinsMined", "TotalCoinSupply"]]
)
clustered_scaled = pd.DataFrame(clustered_scaled, columns=["TotalCoinsMined", "TotalCoinSupply"], index=clustered_df.index)
clustered_scaled["CoinName"]=clustered_df["CoinName"]
clustered_scaled["Class"]=clustered_df["Class"]
clustered_scaled

In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
clustered_scaled.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", hover_cols=["CoinName"], by="Class")